In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import dispvideo as dv

DATADIR = "data/"
fnames = os.listdir(DATADIR)
fnames = [os.path.join(DATADIR, i) for i in fnames]


In [ ]:
tmp = scipy.io.loadmat(fnames[0])
key = [i for i in list(tmp.keys()) if i not in ['__header__', '__version__', '__globals__']][0]
print("key:", key)
mat = tmp[key]
gray = dv.grayscale(mat)
# For convenience...
gray = np.moveaxis(gray, -1, 0)

In [ ]:
gray.shape